<a href="https://colab.research.google.com/github/aithaprasad/related-languages/blob/main/Uday_NLP_5_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We just read the file and return sentences based on the given start and end tags

In [1]:
def get_sentences_from_file(file):
  sentences = []
  temp = []
  data = open(file, 'r')
  for line in data:
    token = line.rstrip("\n")
    if token == '<s>': temp = []
    elif token == '</s>': sentences.append(temp)
    else: temp.append(line.split('\n')[0])
  return sentences

In [2]:
train_source = get_sentences_from_file('train-source.txt')
train_target = get_sentences_from_file('train-target.txt')

In [3]:
test_source = get_sentences_from_file('test-source.txt')
test_target = get_sentences_from_file('test-target.txt')

In [4]:
len(train_source) == len(train_target)

True

In [28]:
train_source, val_source = train_source[:-5000], train_source[-5000:]
train_target, val_target = train_target[:-5000], train_target[-5000:]

This function returns transmission probabilities indicating with what probablity does a particular word get turn into a new word

In [5]:
def source_to_target_words_map(source, target):
  
  transmission_counts, transmission_probs = dict(), dict()
  
  for i in range(len(source)):
    for j in range(min(len(source[i]), len(target[i]))):
      if source[i][j] not in transmission_counts:
        transmission_counts[source[i][j]] = {target[i][j] : 1}
      elif target[i][j] not in transmission_counts[source[i][j]]:
        transmission_counts[source[i][j]][target[i][j]] = 1
      else:
        transmission_counts[source[i][j]][target[i][j]] += 1

  for key, data_dict in transmission_counts.items():
      transmission_probs[key] = {}
      dict_sum = sum(list(data_dict.values()))

      for new_key, value in data_dict.items():
          transmission_probs[key][new_key] = (transmission_counts[key][new_key] / dict_sum)

    
  return transmission_probs

In [6]:
transmission_probabilities = source_to_target_words_map(train_source, train_target)

This predict is almost similar to the viterbi function from my Kreyol segmentation assignment.

In [24]:
def predict(test):
    all_sentences = []
    for sentence in test:
        curr_sentence = []
        for word in sentence:
            if word in transmission_probabilities:
                curr_high_probability, translated_word = 0, ''
                for key in transmission_probabilities[word]:
                    if transmission_probabilities[word][key] > curr_high_probability:
                        curr_high_probability = transmission_probabilities[word][key]
                        translated_word = key
                curr_sentence.append(translated_word)
        all_sentences.append(curr_sentence)
    return all_sentences

In [10]:
from nltk.translate.bleu_score import corpus_bleu

In [25]:
predicted_test_sentences = predict(test_source)

In [27]:
test_target_for_bleu = []
for sentence in test_target:
    test_target_for_bleu.append([sentence])

In [28]:
corpus_bleu(test_target_for_bleu, predicted_test_sentences)

0.7421058136552283